# MOS Operating Point

Reading PSF files from cadence dc sweep simulation

Using psf-utils from K. Kundert at https://github.com/KenKundert/psf_utils/tree/master

Install psf-utils with pip3

--> Using latest downloaded version to accommodate .info files

In [1]:
import os
import sys

In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

In [3]:
import pandas as pd
pd.set_eng_float_format(accuracy=1, use_eng_prefix=True)

In [4]:
import json

In [9]:
from psf_utils import PSF
from inform import Error, display

In [10]:
#check that the new copy of psf_utils is loaded
sys.modules["psf_utils"]

<module 'psf_utils' from '/Users/peterkinget/Library/Mobile Documents/com~apple~CloudDocs/BOX/Python/JupyterNotebooks/MOS_operating_point/psf_utils/__init__.py'>

In [21]:
def psf_list_signals(psf, beginning=""):
    for signal in psf.all_signals():
        # print(signal.name[0:len(beginning)])
        if (signal.name[0:len(beginning)]==beginning):
            print(signal.name, signal.units)

In [45]:
def collect_transistors_data(psf, transistors, signal_names, tor_signal_separator = '.'): 
    transistors_data = {}
    for transistor in transistors:
        transistors_data[transistor] = {}
        for signal in signal_names:
            tor_signal_name = transistor + tor_signal_separator + signal
            transistors_data[transistor][signal] = float(psf.get_signal(tor_signal_name).ordinate)
    return transistors_data

In [15]:
home_folder = '/Users/peterkinget'
project_folder = home_folder + '/files/PK/CircuitDesign/MOS_Transistor_Characterization/tsmc25_public/capacitances'
# results_folder = project_folder + '/dcop_simulation/psf'
results_folder = project_folder + '/dcop_simulation/psf/presized_OTA_tb_v2'
results_folder = "/Users/peterkinget/iCloudDrive/BOX/Python/JupyterNotebooks/MOS_operating_point/presized_OTA_tb_v2"
print(results_folder)

/Users/peterkinget/iCloudDrive/BOX/Python/JupyterNotebooks/MOS_operating_point/presized_OTA_tb_v2


In [16]:
os.chdir(results_folder)
os.listdir()

['.psf_file',
 'element.info.ascii',
 'element.info.ascii.cache',
 'operating_point.csv',
 '.psf_show_args',
 '#element.info.ascii#',
 '.#element.info.ascii',
 'dcOpInfo.info.ascii.cache',
 'dcOp.dc.ascii.cache',
 'dcOpInfo.info.ascii',
 'dcOp.dc.ascii']

In [17]:
# psf = PSF('dc.dc_vg_sweep_vd_0p825.ascii')
# psf = PSF('dc.dc_vg_sweep_vd_0p825.ascii')
psf = PSF('dcOpInfo.info.ascii')

In [18]:
psf_info = PSF("element.info.ascii")

## Element Information

In [23]:
psf_list_signals(psf_info, beginning="I0.M3")

I0.M3.w m
I0.M3.l m
I0.M3.as m^2
I0.M3.ad m^2
I0.M3.ps m
I0.M3.pd m
I0.M3.nrd m/m
I0.M3.nrs m/m
I0.M3.m None
I0.M3.trise C
I0.M3.aforward None
I0.M3.areverse None
I0.M3.delvto V
I0.M3.mulmu0 None
I0.M3.mulu0 None
I0.M3.delk1 sqrt(V)
I0.M3.delnfct None
I0.M3.rdc None
I0.M3.rsc None
I0.M3.sa m
I0.M3.sb m
I0.M3.sa1 m
I0.M3.sa2 m
I0.M3.sa3 m
I0.M3.sa4 m
I0.M3.sa5 m
I0.M3.sa6 m
I0.M3.sa7 m
I0.M3.sa8 m
I0.M3.sa9 m
I0.M3.sa10 m
I0.M3.sb1 m
I0.M3.sb2 m
I0.M3.sb3 m
I0.M3.sb4 m
I0.M3.sb5 m
I0.M3.sb6 m
I0.M3.sb7 m
I0.M3.sb8 m
I0.M3.sb9 m
I0.M3.sb10 m
I0.M3.sw1 m
I0.M3.sw2 m
I0.M3.sw3 m
I0.M3.sw4 m
I0.M3.sw5 m
I0.M3.sw6 m
I0.M3.sw7 m
I0.M3.sw8 m
I0.M3.sw9 m
I0.M3.sw10 m
I0.M3.sca None
I0.M3.scb None
I0.M3.scc V
I0.M3.sc m
I0.M3.mulid0 None
I0.M3.mulbeta None
I0.M3.nqsmod None
I0.M3.acnqsmod None
I0.M3.geo None
I0.M3.stimod None
I0.M3.region None
I0.M3.isnoisy None


## Operating Point Information

In [25]:
psf_list_signals(psf, beginning='I0.M3.v')

I0.M3.vgs V
I0.M3.vds V
I0.M3.vbs V
I0.M3.vgb V
I0.M3.vdb V
I0.M3.vgd V
I0.M3.vth V
I0.M3.vdsat V
I0.M3.vfbeff V
I0.M3.vgsteff V
I0.M3.vsb V
I0.M3.vgt V
I0.M3.vdss V
I0.M3.vsat_marg V
I0.M3.vearly V
I0.M3.vth_drive V
I0.M3.vdsat_marg V


In [26]:
signal_names = []
for signal in psf.all_signals():
    if (signal.name[0:6] == 'I0.M3.'):
        # print(signal.name) #, signal.units, psf.get_signal(signal.name).ordinate)
        signal_names.append(signal.name[6:])

In [24]:
# signal_names

In [27]:
capacitor_names = [ name for name in signal_names if name[0]=='c']

In [29]:
# transistor = "Mn11"
# transistor = "I7.M8"
# for signal in signal_names_op:
#     tor_signal_name = transistor + '.' + signal
#     tor_signal = psf.get_signal(tor_signal_name).ordinate
#     print(tor_signal_name, float(tor_signal), tor_signal.units)

In [30]:
tor_root = "Mn"
transistors = []
for i in range(4):
    for j in range(4):
        transistors.append(tor_root+str(i+1)+str(j+1))
# transistors

In [32]:
devicefile_path = "/Users/peterkinget/iCloudDrive/BOX/Python/JupyterNotebooks/MOS_operating_point/"
devicefile = "device_names.json"
with open(devicefile_path + devicefile) as f:
    json_data = f.read()
transistor_names_shortcut_to_netlist = json.loads(json_data)
transistor_names_shortcuts = transistor_names_shortcut_to_netlist.keys()
transistor_names_netlist = transistor_names_shortcut_to_netlist.values()
transistor_names_netlist_to_shortcut = dict(zip(transistor_names_netlist,transistor_names_shortcuts))

In [33]:
signal_names_op = [ 'ids', 'vgs', 'vds', 'vdsat', 'region',
                    'vbs', 'vth', 'vgsteff', 
                    'gm', 'gds', 'gmb', 'gmoverid', 'self_gain', 
                    'cgg', 'cgs', 'cgd', 'cgb',
                    'csg', 'css', 'csd', 'csb', 
                    'cdg', 'cds', 'cdd', 'cdb', 
                    'cbg', 'cbs', 'cbd', 'cbb',
                    'cjd', 'cjs', 'cgsovl', 'cgdovl', 'cgbovl',
                    'fug']

In [51]:
transistors_data = collect_transistors_data(psf, transistor_names_netlist_to_shortcut.keys(), signal_names_op)

In [52]:
sizing_names = [ 'w', 'l', 'as', 'ad', 'ps', 'pd', 'm']

In [61]:
transistors_sizing = collect_transistors_data(psf_info, transistor_names_netlist_to_shortcut.keys(), sizing_names)

In [62]:
df_info = pd.DataFrame(transistors_sizing).T

In [81]:
df = pd.DataFrame(transistors_data).T
# columns are the parameters
# rows are the transistors

In [66]:
df_info.head()

,w,l,as,ad,ps,pd,m
I7.M1,8.0u,500.0n,4.0p,2.0p,9.0u,8.0p,1.0
I7.M2,8.0u,500.0n,4.0p,2.0p,9.0u,8.0p,1.0
I7.M3,24.0u,500.0n,8.0p,6.0p,27.0u,16.0p,1.0
I7.M4,24.0u,500.0n,8.0p,6.0p,27.0u,16.0p,1.0
I7.M5,96.0u,500.0n,26.0p,24.0p,108.0u,52.0p,1.0


In [67]:
df.head()

,ids,vgs,vds,vdsat,region,vbs,vth,vgsteff,gm,gds,...,cbg,cbs,cbd,cbb,cjd,cjs,cgsovl,cgdovl,cgbovl,fug
I7.M1,99.5u,736.2m,1.3,218.6m,2.0,-513.5m,512.9m,223.2m,771.2u,14.2u,...,-2.7f,-9.6f,-2.2f,14.5f,2.2f,8.4f,3.8f,3.8f,450.6z,4.9G
I7.M2,99.6u,736.5m,1.3,218.7m,2.0,-513.5m,513.0m,223.3m,771.5u,14.2u,...,-2.7f,-9.6f,-2.2f,14.5f,2.2f,8.4f,3.8f,3.8f,450.6z,4.9G
I7.M3,-99.5u,-722.3m,-722.3m,-258.5m,2.0,0.0,-479.6m,243.7m,688.4u,11.1u,...,-7.4f,-10.4f,-8.8f,26.6f,8.8f,0.0,13.7f,13.7f,417.8z,1.4G
I7.M4,-99.6u,-722.3m,-730.5m,-258.5m,2.0,0.0,-479.6m,243.7m,688.9u,11.0u,...,-7.4f,-10.4f,-8.8f,26.6f,8.8f,0.0,13.7f,13.7f,417.8z,1.4G
I7.M5,-453.9u,-730.5m,-1.3,-267.6m,2.0,0.0,-479.3m,252.0m,3.0m,36.6u,...,-29.3f,-42.4f,-31.0f,102.6f,30.9f,0.0,54.8f,54.8f,417.8z,1.6G


In [84]:
df_merged = pd.merge(df_info.T, df.T, how="right")
df_merged = pd.concat([df_info.T, df.T]).T
df = df_merged

In [85]:
# replace transistor netlist names with shortcuts
new_names = [ transistor_names_netlist_to_shortcut[label] for label in df.index]
df.index = new_names

In [86]:
# convert region to words
df['region'] = df['region'].map({0.0:'off', 1.0:'linear', 2.0:'saturation'})

In [87]:
# convert signs of transcapacitance to Tsividis convention
# Cxx = dQx/dVx and Cxy = -dQx/dVy with x <> y
def cap_sign(cap):
    if cap[1] == cap[2]:
        cap_sign = 1
    else:
        cap_sign = -1
    return cap_sign
trans_capacitor_names = [ 'cgs', 'cgd', 'cgb', 'csg', 'csd', 'csb', 'cdg', 'cds', 'cdb',
                          'cbg', 'cbs', 'cbd' ]
for capacitor in trans_capacitor_names:
    df[capacitor] = (-1)*df[capacitor]

In [88]:
# calculate the transcapacitances for the small-signal model (Tsividis Fig. 8.5)
df['cm'] = df['cdg']-df['cgd']
df['cmb'] = df['cdb']-df['cbd']
df['cmx'] = df['cbg']-df['cgb']

In [106]:
element_info_print = ['w', 'l', 'm', 'as', 'ad', 'ps', 'pd' ]

In [107]:
signal_names_op_print = [ 'ids', 'vgs', 'vds', 'vdsat', 'region',
                          'vbs', 'vth', 'vgsteff', 
                          'gm', 'gds', 'gmb', 'gmoverid', 'self_gain', 
                          'cgs', 'cgsovl', 'cgb', 'cgbovl', 'cgd', 'cgdovl',
                          'cbd', 'cjd', 'cbs' , 'cjs',
                          'csd', 'cm', 'cmb', 'cmx','fug']

In [108]:
# print(df[signal_names_op_print].T)
# df[signal_names_op_print].loc[["M1", "M1b"]].T
df[[*element_info_print, *signal_names_op_print]].T

,M1,M2,M3,M4,M5,M6,M7,M8
w,8.0u,8.0u,24.0u,24.0u,96.0u,32.0u,16.0u,8.0u
l,500.0n,500.0n,500.0n,500.0n,500.0n,500.0n,500.0n,500.0n
m,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
as,4.0p,4.0p,8.0p,8.0p,26.0p,10.0p,6.0p,4.0p
ad,2.0p,2.0p,6.0p,6.0p,24.0p,8.0p,4.0p,2.0p
ps,9.0u,9.0u,27.0u,27.0u,108.0u,36.0u,18.0u,9.0u
pd,8.0p,8.0p,16.0p,16.0p,52.0p,20.0p,12.0p,8.0p
ids,99.5u,99.6u,-99.5u,-99.6u,-453.9u,453.9u,199.2u,100.0u
vgs,736.2m,736.5m,-722.3m,-722.3m,-730.5m,644.9m,644.9m,644.9m
vds,1.3,1.3,-722.3m,-730.5m,-1.3,1.2,513.5m,644.9m


In [450]:
csv_filename = "operating_point.csv"
df[signal_names_op_print].T.to_csv(devicefile_path+csv_filename)

In [383]:
# df.iloc[:,0:14]
# df.iloc[:, 14:]

In [396]:
df.index

Index(['I7.M1', 'I7.M2', 'I7.M3', 'I7.M4', 'I7.M5', 'I7.M6', 'I7.M7', 'I7.M8',
       'I0.M1', 'I0.M2', 'I0.M3', 'I0.M4', 'I0.M5', 'I0.M6', 'I0.M7', 'I0.M8'],
      dtype='object')

In [384]:
df['gm']/df['cm']/2/np.pi

I7.M1    16.0G
I7.M2    16.0G
I7.M3     5.3G
I7.M4     5.3G
I7.M5     5.8G
I7.M6    16.8G
I7.M7    15.5G
I7.M8    15.7G
dtype: float64

## OTA calculations

In [271]:
Adc1 = df.loc["I7.M2"]['gm']/(df.loc["I7.M2"]['gds']+df.loc["I7.M4"]['gds'])
Adc1_dB = 20*np.log10(Adc1)
Adc2 = df.loc["I7.M5"]['gm']/(df.loc["I7.M5"]['gds']+df.loc["I7.M6"]['gds'])
Adc2_dB = 20*np.log10(Adc2)
print(Adc1, Adc1_dB, Adc2, Adc2_dB, Adc1_dB+Adc2_dB)

30.512856007182435 29.689657191267955 30.939902094810858 29.810378701972425 59.50003589324038


In [275]:
Cc = 330e-12
fu = df.loc["I7.M1"]['gm']/2/np.pi/Cc

In [290]:
Cl = 880e-12
fnd = df.loc["I7.M5"]['gm']/2/np.pi/Cl

In [294]:
Rc = 740
fz = 1/(2*np.pi*(1/df.loc["I7.M5"]['gm']-Rc)*Cc)

In [295]:
fmir = df.loc["I7.M3"]['gm']/2/np.pi/(df.loc["I7.M3"]["cgs"]+df.loc["I7.M4"]["cgs"]+df.loc["I7.M3"]["cdb"])

In [296]:
print(f"ADC {Adc1_dB+Adc2_dB:.3f} fu {fu:.3e} fnd {fnd:.3e} fz {fz:.3e} fmir {fmir:.3e}")

ADC 59.500 fu 3.719e+05 fnd 5.469e+05 fz -1.178e+06 fmir 8.401e+08
